**SetupDemo.ipynb**

Written by Taiob Ali  
SqlWorldWide.com

This script will create:

- A resource group
- 1 logical SQL server
- 1 sample databases using BACPAC located at [GitHub](https://github.com/microsoft/sql-server-samples/blob/master/samples/features/query-store/PropertyMLS.bacpac)

Install the Azure PowerShell module if you have not already

In [ ]:
Install-Module -Name Az

If you've disabled module autoloading, manually import the module with `Import-Module -Name Az`. Because of the way the module is structured, this can take a few seconds.

In [ ]:
Import-Module -Name Az

Sign in to Azure

In [ ]:
Connect-AzAccount

If you need to see the list of your subscription

<span style="color: #6a9955;">$SubscriptionList=Get-AzSubscription</span><span style="color: rgb(106, 153, 85);">$SubscriptionList</span>

Use below code if you have multiple subscription and you want to use a particular one

In [ ]:
Set-AzContext -SubscriptionId '18d92f52-ac34-4379-ab8b-5a5106f1c54e'

Declare Variables

In [ ]:
$resourceGroupName = "dataweekenderdemo"
$primaryLocation = "East US 2" 
$serverName = "qshints"
$databaseName = "PropertyMLS"
$ipinfo = Invoke-RestMethod http://ipinfo.io/json 
$startip = $ipinfo.ip
$endip = $ipinfo.ip 
$adminlogin = "taiob"
$password = Get-Content "C:\password.txt"

Check if Resource group exists. If exist delete the resource group.

Create a new Resource group

In [ ]:
$resGrpChk = Get-AzResourceGroup `
    -Name $resourceGroupName `
    -ev notPresent `
    -ea 0

if ($resGrpChk) {  
    #Delete resource group
    Remove-AzResourceGroup `
        -Name $resourceGroupName -Confirm   
    Write-Host 'Resource group deleted' `
        -fore white `
        -back green
}

New-AzResourceGroup `
    -Name $resourceGroupName `
    -Location "East US 2"    
Write-Host 'Resource group created' `
    -fore white `
    -back green

Create a Azure SQL Server at East US 2

In [ ]:
New-AzSqlServer `
  -ResourceGroupName $resourceGroupName `
  -ServerName $serverName `
  -Location $primaryLocation `
  -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
  -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force)) 

Configure Firewall rule for the above server

In [ ]:
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $serverName `
  -FirewallRuleName "TaiobDemoMachine" `
  -StartIpAddress $startip `
  -EndIpAddress $endip

# This is done to allow access to Azure Services
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $serverName `
  -FirewallRuleName "AllowAzureService" `
  -StartIpAddress 0.0.0.0 `
  -EndIpAddress 0.0.0.0

Creating an Azure SQL Database with using BACPAC located at [GitHub](https://github.com/microsoft/sql-server-samples/blob/master/samples/features/query-store/PropertyMLS.bacpac)

Puchase Model: vCore  
Service Tier : GeneralPurpose

Note:

1. Downloading bacpac file using powershell is corrupting the file, I could not figure out the reason. So I downloaded manually to my local drive. 
2. More about : [Import a BACPAC file to a database in Azure SQL Database or Azure SQL Managed Instance](https://docs.microsoft.com/en-us/azure/azure-sql/database/database-import?tabs=azure-powershell)

In [ ]:
$bacpacfile = "C:\Users\taiob\Downloads\PropertyMLS.bacpac"
Set-Location "C:\Program Files (x86)\Microsoft SQL Server\140\DAC\bin\"
.\sqlpackage.exe /a:Import /sf:$bacpacfile /tsn:"$serverName.database.windows.net" /tdn:$databaseName  /tu:$adminlogin /tp:$password  /p:DatabaseServiceObjective="GP_Gen5_4"

Clean up by removing resource group name

In [ ]:
Remove-AzResourceGroup -ResourceGroupName $resourceGroupName -Force